# aiapy: A SunPy affiliated package for analyzing data from the Atmospheric Imaging Assembly (AIA)

## aiapy showcase

`aiapy` is a SunPy affiliated package for analyzing EUV image data from the Atmospheric Imaging Assembly (AIA).
It includes a number of of functions for calibrating images, correcting metadata, and analyzing the filter bandpasses.
In this post, we show some examples of the different capabilities of the `aiapy` package.

Resources
---------------

- [Source code](https://gitlab.com/LMSAL_HUB/aia_hub/aiapy)
- [Documentation](https://aiapy.readthedocs.io/en/stable/)
- [JOSS paper](https://joss.theoj.org/papers/10.21105/joss.02801)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sunpy.net import Fido, attrs as a
import sunpy.map
import astropy.units as u
from astropy.coordinates import SkyCoord
import astropy.time
from astropy.visualization import time_support

from aiapy.calibrate import (register,update_pointing,correct_degradation,
                             degradation,normalize_exposure, respike, fetch_spikes)

from aiapy.response import Channel

import matplotlib as mpl
# Increases the figure size in this notebook.
mpl.rcParams["savefig.dpi"] = 150
mpl.rcParams["figure.dpi"] = 150

Query 2-3 AIA images from different wavelengths and create [`sunpy.map.Map`](https://docs.sunpy.org/en/stable/code_ref/map.html) objects.

In [ ]:
t_start = astropy.time.Time('2012-01-01T00:00:00')
search_results = Fido.search(
    a.Time(t_start, t_start+11*u.s),
    a.Instrument('AIA'),
    a.Wavelength(171*u.angstrom) | a.Wavelength(335*u.angstrom),
)
search_results

In [ ]:
files = Fido.fetch(search_results,max_conn=1)

In [ ]:
m_171, m_335 = sunpy.map.Map(sorted(files))

## Respiking Level 1 Images

Respiking an AIA image is quite straightfoward.

In [ ]:
m_171_respiked = respike(m_171)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(1, 2, 1, projection=m_171)
m_171.plot(axes=ax1)

ax2 = fig.add_subplot(1, 2, 2, projection=m_171)
m_171_respiked.plot(axes=ax2)

ax1.set_title("Original Level 1")
ax2.set_title("Respiked Image")
ax1.set_axis_off()
ax2.set_axis_off()

If we histogram the intensity values before and after, the changes very obvious.

In [ ]:
pix, vals = fetch_spikes(m_171)

In [ ]:
plt.hist(m_171.data[pix.y.value.round().astype(int),
                    pix.x.value.round().astype(int)].flatten(), log=True, bins='scott', histtype='step', label='Despiked');
plt.hist(vals, log=True, bins='scott', histtype='step', label='Respiked');
plt.legend()
plt.xlabel('Intensity [DN]')
plt.ylabel('Frequency')

If we want to get the locations of the spikes, we can do the following:

In [ ]:
m_171_cutout = m_171.submap(
    SkyCoord(-600*u.arcsec, -600*u.arcsec, frame=m_171.coordinate_frame),
    top_right=SkyCoord(100*u.arcsec, 100*u.arcsec, frame=m_171.coordinate_frame),
)

In [ ]:
spike_coords, _ = fetch_spikes(m_171_cutout, as_coords=True)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1, projection=m_171_cutout)
ax.plot_coord(spike_coords, 'o', color='C0', fillstyle='none', markersize=5)
m_171_cutout.plot(axes=ax)

## Transforming Level 1 Images to Level 1.5

If you want to use lots of AIA images and compare them to each other, you will need to create level 1.5 data.

We can do the following steps to do that:

1. Update the pointing

In [ ]:
m_171.reference_pixel

In [ ]:
m_171 = update_pointing(m_171)

In [ ]:
m_171.reference_pixel

2. Image registration 

In [ ]:
m_171.scale

In [ ]:
m_171.rotation_matrix

In [ ]:
m_171_lvl15 = register(m_171)

Note that you can combine these two operations into a "prep" function to imitate `aiaprep.pro`

In [ ]:
def prep(smap):
    return register(update_pointing(smap))

In [ ]:
m_335_lvl15 = prep(m_335)

In [ ]:
print(m_171_lvl15.scale)
print(m_335_lvl15.scale)

In [ ]:
print(m_171_lvl15.rotation_matrix)
print(m_335_lvl15.rotation_matrix)

## Degradation Correction

As all things do in life do, they age, fail and deteriorate until they die.

While we can't fix death, we can fix AIA filter degradation.

The function `degradation`, will allow you pass in a filter wavelength and a time period.

In [ ]:
t_begin = astropy.time.Time('2010-03-25T00:00:00')
now = astropy.time.Time.now()
time_window = t_begin + np.arange(0, (now - t_begin).to(u.day).value, 7) * u.day

d_335 = degradation(335*u.angstrom, time_window)

We can also look at older versions of the calibration to see how the predicted degradation has changed. Note that the default version in `v0.6.0` of aiapy is 9.

In [ ]:
d_335_v8 = degradation(335*u.angstrom, time_window, calibration_version=8)

And compare the two degradation curves as a function of time over the lifetime of the mission.

In [ ]:
with time_support(format='jyear'):
    plt.plot(t,d_335,label='v9')
    plt.plot(t,d_335_v8,label='v8')
    plt.plot(m_335.date[np.newaxis], degradation(335*u.angstrom, m_335.date),
             linestyle='', marker='.', color='C0', markersize=15, label=m_335.date)
plt.ylabel('Degradation 335 $\mathrm{\AA}$')
plt.legend()

Note that we can pass an image into the `correct_degradation` function in order to correct the whole image.

In [ ]:
m_335_corrected = correct_degradation(m_335)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(121,projection=m_335)
m_335.plot(axes=ax,vmin=0,vmax=200,title='Uncorrected')
ax.set_axis_off()
ax = fig.add_subplot(122,projection=m_335_corrected)
m_335_corrected.plot(axes=ax,vmin=0,vmax=100,title='Corrected')
ax.set_axis_off()

## Exposure Time Normalization

A common process is to normalize the counts on the exposure time.

In [ ]:
m_171_lvl15.exposure_time

In [ ]:
m_171_norm = normalize_exposure(m_171_lvl15)

In [ ]:
m_171_norm.exposure_time

## Wavelength Response Functions

We can calculate the wavelength response for a single channel, as well as the components of it, and then show all channels.

In [ ]:
c = Channel(m_335.wavelength)

In [ ]:
r = c.wavelength_response()
r_time = c.wavelength_response(obstime=m_335.date)
r_time_eve = c.wavelength_response(obstime=m_335.date, include_eve_correction=True)

In [ ]:
plt.plot(c.wavelength,r,label='Uncorrected')
plt.plot(c.wavelength,r_time,label='Time Correction')
plt.plot(c.wavelength,r_time_eve,label='Time + EVE Correction')
plt.xlim([315,355])
plt.ylim([0,0.03])
plt.xlabel('Wavelength [$\mathrm{\AA}$]')
plt.ylabel(f'Wavelength Response [{r.unit.to_string(format="latex")}]')
plt.legend(frameon=False)

We hope this showcase was really helpful!
`aiapy` is under development and we would appreciate users, feedback and feature requests on [GitLab](https://gitlab.com/LMSAL_HUB/aia_hub/aiapy/-/issues).